<a href="https://colab.research.google.com/github/hucarlos08/GEO-ML/blob/main/KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KNN

**k Vecinos más Cercanos (KNN)**

**Objetivo:** Clasificar un objeto no etiquetado basándose en la clasificación de sus \( k \) vecinos más cercanos.

**Procedimiento:**
1. Dado un punto de consulta $x_{\text{query}}$, calcula la distancia entre $x_{\text{query}}$ y cada punto en el conjunto de datos.
2. Ordena las distancias en orden ascendente y selecciona las $k$ más pequeñas.
3. Usa la etiqueta más común entre las $k$ observaciones más cercanas como la etiqueta predicha para $x_{\text{query}}$.

**Ecuaciones clave:**

1. **Distancia Euclidiana** (para datos de 2 dimensiones, fácilmente extensible a más dimensiones):
$d(p, q) = \sqrt{(p_1 - q_1)^2 + (p_2 - q_2)^2}$

2. **Decisión de Clasificación**:
$y_{\text{pred}} = \text{moda}\left(y_{i1}, y_{i2}, \ldots, y_{ik}\right) $
Donde $ y_{i1}, y_{i2}, \ldots, y_{ik} $ son las etiquetas de los \( k \) vecinos más cercanos.

**Notas:**
- La elección de \( k \) es crucial. Un \( k \) pequeño puede ser sensible al ruido, mientras que un \( k \) grande puede suavizar demasiado las decisiones de clasificación.
- KNN puede usarse tanto para clasificación como para regresión. Para regresión, en lugar de tomar la moda de las etiquetas de los vecinos, se toma el promedio.
- La función de distancia utilizada puede variar. Aunque la distancia euclidiana es común, otras distancias como la Manhattan, Minkowski o Mahalanobis también pueden ser apropiadas dependiendo del contexto.


In [3]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from sklearn.datasets import make_blobs
from sklearn.neighbors import NearestNeighbors
import numpy as np

from IPython.display import HTML

# Generar datos sintéticos bidimensionales
np.random.seed(0)
X, y = make_blobs(n_samples=200, centers=4, cluster_std=1.0, center_box=(-10.0, 10.0))

# Punto de consulta (lo colocaremos cerca del centro de los datos para una mejor visualización)
query_point = np.array([[0, 0]])

# Función para obtener los k vecinos más cercanos
def get_neighbors(X, query_point, k):
    nbrs = NearestNeighbors(n_neighbors=k).fit(X)
    _, indices = nbrs.kneighbors(query_point)
    return X[indices]

# Configurar la figura y el eje
fig, ax = plt.subplots(figsize=(8, 8))
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='viridis', s=30)
point, = plt.plot(query_point[0, 0], query_point[0, 1], 'ro', markersize=10)
neighbors, = plt.plot([], [], 'go', markersize=6, alpha=0.6)
circle = plt.Circle((query_point[0, 0], query_point[0, 1]), 0, fill=False, color='r', linestyle='dashed')
ax.add_artist(circle)

# Función de inicialización para la animación
def init():
    neighbors.set_data([], [])
    circle.radius = 0
    return neighbors, circle

# Función de actualización para la animación
def update(k):
    if k == 0:  # reiniciar para k=0
        return init()

    neigh_points = get_neighbors(X, query_point, k)
    neighbors.set_data(neigh_points[0][:, 0], neigh_points[0][:, 1])

    # Ajustar el radio del círculo para coincidir con el k-ésimo vecino más lejano
    last_neighbor = neigh_points[0][-1]
    circle.radius = np.sqrt((last_neighbor[0] - query_point[0, 0])**2 + (last_neighbor[1] - query_point[0, 1])**2)

    return neighbors, circle

# Crear animación
ani = FuncAnimation(fig, update, frames=range(0, 31), init_func=init, blit=True, repeat=True)

plt.close()
HTML(ani.to_html5_video())

## Las métricas

In [5]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# 1. Generar conjunto de datos sintético
X, y = make_classification(n_samples=500, n_features=20, n_redundant=5, random_state=42)

# 2. Dividir conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 3. Entrenar y evaluar modelo KNN con diferentes métricas de distancia
metrics = ["euclidean", "manhattan", "minkowski", "chebyshev"]
accuracies = []

for metric in metrics:
    knn = KNeighborsClassifier(n_neighbors=5, metric=metric)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracies.append(accuracy_score(y_test, y_pred))

for acc, met in zip(accuracies,metrics):
  print(met, acc)


euclidean 0.86
manhattan 0.8933333333333333
minkowski 0.86
chebyshev 0.7933333333333333
